# IMPORT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='ticks', palette='RdBu')
#sns.set(style='ticks', palette='Set2')
import datetime
import pytz
import businesstimedelta
import holidays as pyholidays
import time as t
from workalendar.africa import Kenya
from datetime import date
%matplotlib inline

In [2]:
#3rd Floor All user Updates_1

status = pd.read_excel(r"C:\Users\USER\Documents\Alex\LENS\3rdfloor.xlsx")

In [3]:
status.head(1)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status
0,approvals1,25.03.23,12:07:00,Draft Payments Posted,1002019,235401413,Frame from HQ and Lens from HQ


In [4]:
status["Date"]=pd.to_datetime(status["Date"],dayfirst = True).dt.date

In [5]:
status['Datetime']=status['Date'].astype(str)+' '+status['Time'].astype(str)

In [6]:
status.head(1)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime
0,approvals1,2023-03-25,12:07:00,Draft Payments Posted,1002019,235401413,Frame from HQ and Lens from HQ,2023-03-25 12:07:00


In [7]:
status['Datetime']=pd.to_datetime(status['Datetime'], format='%Y/%m/%d %H:%M:%S')

# DesignerStore

## DESIGNER(rejected frame sent to frame store to reissued frame for order)-designerdamage_reissued

In [8]:
designerrejected = status[status['Status']== 'Rejected Frame sent to Frame Store']
designerrejected['Status'].unique()

array([], dtype=object)

In [9]:
designerrejected1 = designerrejected[status['Created User']== 'control1']
designerrejected2 = designerrejected[status['Created User']== 'control2']

designerrejected=pd.concat([designerrejected1,designerrejected2])

designerrejected['Status'].unique()

C:\Users\user\AppData\Local\Temp\ipykernel_10048\314140570.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  designerrejected1 = designerrejected[status['Created User']== 'control1']
C:\Users\user\AppData\Local\Temp\ipykernel_10048\314140570.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  designerrejected2 = designerrejected[status['Created User']== 'control2']


array([], dtype=object)

In [10]:
designerrejected['Created User'].unique()

array([], dtype=object)

In [11]:
reissuedframedes = status[status['Status']== 'ReIssued Frame for Order']
reissuedframedes['Status'].unique()

array([], dtype=object)

In [12]:
reissuedframedes1 = status[status['Created User']== 'designer1']
reissuedframedes2 = status[status['Created User']== 'designer2']
reissuedframedes=pd.concat([reissuedframedes1,reissuedframedes2])
reissuedframedes['Created User'].unique()

array(['designer1'], dtype=object)

In [13]:
reissuedframedes.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime
1166,designer1,2023-03-25,09:08:00,Order Printed,1000930,236000677,Frame from HQ and Lens from HQ,2023-03-25 09:08:00
1167,designer1,2023-03-25,09:08:00,Order Printed,1001657,236700291,Frame from HQ and Lens from HQ,2023-03-25 09:08:00
1168,designer1,2023-03-25,09:08:00,Order Printed,1001249,231101838,Overseas Lens with HQ Frame,2023-03-25 09:08:00


In [14]:
reissuedframedes.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [15]:
reissueframedes= pd.merge(designerrejected,reissuedframedes[['DocNum','Datetimeout']], on='DocNum', how='left')
reissueframedes.head(3)

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout


Date time workdays

In [16]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=reissueframedes.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)



C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [17]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=reissueframedes.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [18]:

reissueframedes["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

reissueframedes =reissueframedes.replace("", np.nan)
reissueframedes=reissueframedes.dropna()

In [19]:
reissueframedes['delayed dept']=reissueframedes['delay'].apply(lambda x: 'Delayed at Designer Store' if x>15 else 'Did not delay at Designer Store' )
reissueframedes

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout,delay,delayed dept


In [20]:
designerdamage_reissued=reissueframedes

In [21]:
designerdamage_reissued.head(5)

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout,delay,delayed dept


# MAINSTORE

## MAINSTORE(rejected frame sent to frame store to reissued frame for order)-mainstoredamage_reissued

In [22]:
mainrejected = status[status['Status']== 'Rejected Frame sent to Frame Store']
mainrejected['Status'].unique()

array([], dtype=object)

In [23]:
mainrejected1 = mainrejected[status['Created User']== 'control1']
mainrejected2 = mainrejected[status['Created User']== 'control2']

mainrejected=pd.concat([mainrejected1,mainrejected2])

mainrejected['Status'].unique()

C:\Users\user\AppData\Local\Temp\ipykernel_10048\2446686935.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mainrejected1 = mainrejected[status['Created User']== 'control1']
C:\Users\user\AppData\Local\Temp\ipykernel_10048\2446686935.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mainrejected2 = mainrejected[status['Created User']== 'control2']


array([], dtype=object)

In [24]:
mainrejected['Created User'].unique()

array([], dtype=object)

In [25]:
reissuedframemain = status[status['Status']== 'ReIssued Frame for Order']
reissuedframemain['Status'].unique()

array([], dtype=object)

In [26]:
reissuedframemain1 = status[status['Created User']== 'main1']
reissuedframemain2 = status[status['Created User']== 'main2']
reissuedframemain=pd.concat([reissuedframemain1,reissuedframemain2])
reissuedframemain['Created User'].unique()

array(['main2'], dtype=object)

In [27]:
reissuedframemain.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime
2545,main2,2023-03-25,09:03:00,Order Printed,1002106,232802028,Frame from HQ and Lens from HQ,2023-03-25 09:03:00
2546,main2,2023-03-25,09:04:00,Frame Sent to Lens Store,1002106,232802028,Frame from HQ and Lens from HQ,2023-03-25 09:04:00
2547,main2,2023-03-25,09:10:00,Order Printed,999371,234100922,Frame from HQ and Lens from HQ,2023-03-25 09:10:00


In [28]:
reissuedframemain.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

In [29]:
reissueframemain= pd.merge(mainrejected,reissuedframemain[['DocNum','Datetimeout']], on='DocNum', how='left')
reissueframemain.head(3)

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout


Date time workdays

In [30]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=reissueframemain.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)



C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [31]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=reissueframemain.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [32]:

reissueframemain["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

reissueframemain =reissueframemain.replace("", np.nan)
reissueframemain=reissueframemain.dropna()

In [33]:
reissueframemain['delayed dept']=reissueframemain['delay'].apply(lambda x: 'Delayed at Main store' if x>15 else 'Did not delay at Main store' )
reissueframemain

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout,delay,delayed dept


In [34]:
mainstoredamage_reissued=reissueframemain

In [35]:
mainstoredamage_reissued.head(5)

,Created User,Date,Time,Status,DocEntry,OrderCriteria Status,Datetime,DocNum,Datetimeout,delay,delayed dept


# LENSSTORE

## LENSSTORE(rejected lens sent to Lens store to reissued Lens for order)-lenstoredamage_reissued

In [36]:
lensrejected = status[status['Status']== 'Rejected Lenses sent to Lens Store']
lensrejected['Status'].unique()

array(['Rejected Lenses sent to Lens Store'], dtype=object)

In [37]:
lensrejected1 = lensrejected[status['Created User']== 'control1']
lensrejected2 = lensrejected[status['Created User']== 'control2']

lensrejected=pd.concat([lensrejected1,lensrejected2])

lensrejected['Status'].unique()

C:\Users\user\AppData\Local\Temp\ipykernel_10048\53500968.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lensrejected1 = lensrejected[status['Created User']== 'control1']
C:\Users\user\AppData\Local\Temp\ipykernel_10048\53500968.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lensrejected2 = lensrejected[status['Created User']== 'control2']


array(['Rejected Lenses sent to Lens Store'], dtype=object)

In [38]:
lensrejected['Created User'].unique()

array(['control1', 'control2'], dtype=object)

In [39]:
reissuedlens = status[status['Status']== 'ReIssued Lens for Order']
reissuedlens['Status'].unique()

array(['ReIssued Lens for Order'], dtype=object)

In [40]:
reissuedlens.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime
1783,lens1,2023-03-25,09:47:00,ReIssued Lens for Order,1001697,23107179,Frame from HQ and Lens from HQ,2023-03-25 09:47:00
1822,lens1,2023-03-25,10:39:00,ReIssued Lens for Order,1001883,23902417,Frame from HQ and Lens from HQ,2023-03-25 10:39:00
1833,lens1,2023-03-25,10:59:00,ReIssued Lens for Order,1001789,232502882,Surfacing Lens with Branch Frame,2023-03-25 10:59:00


In [41]:
reissuedlens.rename(columns={'Datetime':'Datetimeout'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_10048\228283176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reissuedlens.rename(columns={'Datetime':'Datetimeout'}, inplace=True)


In [42]:
reissuedlens= pd.merge(lensrejected,reissuedlens[['DocNum','Datetimeout']], on='DocNum', how='left')
reissuedlens.head(3)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout
0,control1,2023-03-25,09:43:00,Rejected Lenses sent to Lens Store,1001697,23107179,Frame from HQ and Lens from HQ,2023-03-25 09:43:00,2023-03-25 09:47:00
1,control1,2023-03-25,11:32:00,Rejected Lenses sent to Lens Store,1000718,235900156,Order with PF from Branch and Lens from HQ,2023-03-25 11:32:00,2023-03-25 11:24:00
2,control2,2023-03-25,10:27:00,Rejected Lenses sent to Lens Store,1001789,232502882,Surfacing Lens with Branch Frame,2023-03-25 10:27:00,2023-03-25 10:59:00


Date time workdays

In [43]:
##Define a working day

####Days of the week
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(9),
    end_time=datetime.time(18),
    working_days=[0,1, 2, 3, 4])

cal = Kenya()
hl = cal.holidays()
vic_holidays = pyholidays.KE() 
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([workday, holidays], hl)

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""
    
RejectedSentWk_hrs=reissuedlens.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)



C:\Users\user\anaconda3\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [44]:
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        ""

RejectedSentSat_hrs=reissuedlens.apply(lambda row: BusHrs(row['Datetime'], row['Datetimeout']), axis=1)


In [45]:

reissuedlens["delay"]=(RejectedSentWk_hrs+RejectedSentSat_hrs)*60

reissuedlens =reissuedlens.replace("", np.nan)
reissuedlens=reissuedlens.dropna()

In [46]:
reissuedlens['delayed dept']=reissuedlens['delay'].apply(lambda x: 'Delayed at Lens store' if x>15 else 'Did not delay at Lens store' )
reissuedlens

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,control1,2023-03-25,09:43:00,Rejected Lenses sent to Lens Store,1001697,23107179,Frame from HQ and Lens from HQ,2023-03-25 09:43:00,2023-03-25 09:47:00,4.0,Did not delay at Lens store
2,control2,2023-03-25,10:27:00,Rejected Lenses sent to Lens Store,1001789,232502882,Surfacing Lens with Branch Frame,2023-03-25 10:27:00,2023-03-25 10:59:00,32.0,Delayed at Lens store
3,control2,2023-03-25,10:28:00,Rejected Lenses sent to Lens Store,1001883,23902417,Frame from HQ and Lens from HQ,2023-03-25 10:28:00,2023-03-25 10:39:00,11.0,Did not delay at Lens store
4,control2,2023-03-25,11:07:00,Rejected Lenses sent to Lens Store,1000718,235900156,Order with PF from Branch and Lens from HQ,2023-03-25 11:07:00,2023-03-25 11:24:00,17.0,Delayed at Lens store
5,control2,2023-03-25,12:04:00,Rejected Lenses sent to Lens Store,1000420,23211242,Frame from HQ and Lens from HQ,2023-03-25 12:04:00,2023-03-25 13:15:00,71.0,Delayed at Lens store
6,control2,2023-03-25,12:31:00,Rejected Lenses sent to Lens Store,999231,23211083,Surfacing Lens with PF,2023-03-25 12:31:00,2023-03-25 12:35:00,4.0,Did not delay at Lens store
7,control2,2023-03-25,13:38:00,Rejected Lenses sent to Lens Store,1002021,23211471,Order with New Frame from Branch and Lens from HQ,2023-03-25 13:38:00,2023-03-25 13:41:00,3.0,Did not delay at Lens store


In [47]:
lenstoredamage_reissued=reissuedlens

In [48]:
lenstoredamage_reissued.head(5)

,Created User,Date,Time,Status,DocEntry,DocNum,OrderCriteria Status,Datetime,Datetimeout,delay,delayed dept
0,control1,2023-03-25,09:43:00,Rejected Lenses sent to Lens Store,1001697,23107179,Frame from HQ and Lens from HQ,2023-03-25 09:43:00,2023-03-25 09:47:00,4.0,Did not delay at Lens store
2,control2,2023-03-25,10:27:00,Rejected Lenses sent to Lens Store,1001789,232502882,Surfacing Lens with Branch Frame,2023-03-25 10:27:00,2023-03-25 10:59:00,32.0,Delayed at Lens store
3,control2,2023-03-25,10:28:00,Rejected Lenses sent to Lens Store,1001883,23902417,Frame from HQ and Lens from HQ,2023-03-25 10:28:00,2023-03-25 10:39:00,11.0,Did not delay at Lens store
4,control2,2023-03-25,11:07:00,Rejected Lenses sent to Lens Store,1000718,235900156,Order with PF from Branch and Lens from HQ,2023-03-25 11:07:00,2023-03-25 11:24:00,17.0,Delayed at Lens store
5,control2,2023-03-25,12:04:00,Rejected Lenses sent to Lens Store,1000420,23211242,Frame from HQ and Lens from HQ,2023-03-25 12:04:00,2023-03-25 13:15:00,71.0,Delayed at Lens store


# EXPORT

In [49]:
import xlsxwriter

In [50]:

with pd.ExcelWriter(r"C:\Users\USER\Documents\Alex\DAMAGE\NEWDAMAGESUPPLYTIMERESULTS.xlsx", engine='xlsxwriter') as writer:
    designerdamage_reissued.to_excel(writer, sheet_name='designer',index=False)
    mainstoredamage_reissued.to_excel(writer, sheet_name='mainstore',index=False)
    lenstoredamage_reissued.to_excel(writer, sheet_name='lensstore',index=False)

writer.save()    

C:\Users\user\AppData\Local\Temp\ipykernel_10048\1399644388.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\user\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
